# Lecture 9



In [2]:
%notebook inline
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import HTML
from ipywidgets import interact, interactive, fixed, interact_manual
from matplotlib import animation, rc
#sns.set_context('poster')
from scipy.stats import poisson
from scipy.stats import norm
from scipy.optimize import minimize
from scipy.stats import chi2

# Differentiation

In [4]:
def fx(x):
      return (x)**3

def dfxdx(x):
      return 3*x**2

In [8]:
@interact(h=widgets.FloatSlider(min=0, max=1, step=0.01, value=0.5,layout=widgets.Layout(width='60%')) , 
          x0=widgets.FloatSlider(min=-5, max=5, step=0.01, value=0.5,layout=widgets.Layout(width='60%')) , continuous_update=True )
def plot(h,x0):
    fig, axs = plt.subplots(1,1,figsize=(16,12
                              ))
    y0 = fx(x0)
    x = np.linspace(-10,10,400)
    y = fx(x)
    plt.plot(x,y,'r--')
    plt.scatter(x0,y0)

    x1 = x0 + h
    x2 = x0 - h
    y1 = fx(x1)
    y2 = fx(x2)
    plt.scatter(x1,y1)
    plt.scatter(x2,y2)
    numericalslope = (y1-y2)/(x1-x2)
    #plt.plot([x2,x0,x1],[y2,y0,y1])
    #plt.plot([x2,x1],[y2,y1])
    #plt.plot([x0-h, x0, x0+h],[y0-h*(y1-y2)/(x1-x2), y0, y0 +h*(y1-y2)/(x1-x2)],linewidth = 3)

    plt.plot([x2,x1],[y2,y1])
    plt.plot([x2,x1],[y2,y2])
    plt.plot([x1,x1],[y2,y1])

    #plt.plot([x0-h, x0+h],[y0-h*(y1-y2)/(x1-x2), y0-h*(y1-y2)/(x1-x2),],linewidth = 3)
    #plt.plot([x0+h, x0+h],[y0-h*(y1-y2)/(x1-x2), y0+h*(y1-y2)/(x1-x2),],linewidth = 3)
    yup = y1+(y1-y2)
    xmin = x2-(x1-x2)
    ymin = y.min()
    if ymin > y2:
        ymin = y2
    plt.ylim(y2-(y1-y2),y1+(y1-y2))
    plt.xlim(x2-(x1-x2),x1+(x1-x2))
    plt.xlabel('x',fontsize=16)
    plt.ylabel('y',fontsize=16)
    # plt.text(1.1,yup*0.1,'$2h$',fontsize=16)
    # plt.text(1.3,yup*0.1,'$f(x_0+h)-f(x_0-h)$',fontsize=16)
    plt.xticks(fontsize=40)
    plt.yticks(fontsize=40)
    
    position = yup*0.8
    position2 = yup*0.6
    if x0 < 0:
      position = yup + 0.5*(y2-y1)
      position2 = yup + (y2-y1)
    plt.text(xmin+0.2*(x1-x2),position, "Slope = {:4.3f}".format(dfxdx(x0)),fontsize=30)
    plt.text(xmin+0.2*(x1-x2),position2, "Numerical = {:4.3f}".format(numericalslope),fontsize=30)

interactive(children=(FloatSlider(value=0.5, description='h', layout=Layout(width='60%'), max=1.0, step=0.01),…

# Numerical Integral with Trapezoid and Simponson's 1/3 rule

In [10]:
@interact(N=widgets.IntSlider(min=1, max=20, step=1, value=1,layout=widgets.Layout(width='60%')) , Nfunc=widgets.IntSlider(min=1, max=5, step=1, value=1,layout=widgets.Layout(width='60%')) , continuous_update=True )
def plot(N, Nfunc):

  def polynew(x,Nfunc):
    if Nfunc == 1:
      return np.sin(x)*np.log(x)
    elif Nfunc == 2:
      return np.exp(np.sin(x)*np.log(x))
    elif Nfunc == 3:
      return np.cos(0.001*(np.sinh(x)*np.log(x)-1)+5)
    else:
      return np.sinh(x)*np.log(x)-1
  fig, axs = plt.subplots(2,2,figsize=(16,12
                              ))
  plt.subplot(211)

  integral=0
  x=np.linspace(1,10,N+1)
  for xi in x:
    if xi == x[-1]:
      continue
    h = (x[1]-x[0])*0.5
    x1 = xi
    x2 = xi+h
    x3 = xi+2*h
    xvalue= [x1,x2,x3]
    yvalue = polynew([x1,x2,x3],Nfunc) 
    integral += (1/3.)*( yvalue[0] + 4*yvalue[1] + yvalue[2])*h
    plt.scatter(xvalue,yvalue,s=300)
   # print(xvalue,yvalue)
    p_fitted = np.polynomial.Polynomial.fit(xvalue, yvalue, deg=2)
    c=p_fitted.convert().coef
    c=np.flip(c)
    xifine = np.linspace(x1,x3,100)
    yifine = np.polyval(c,xifine)
    plt.plot(xifine,yifine)


  xfine=np.linspace(1,10,91)
  yfine = polynew(xfine,Nfunc)
  plt.plot(xfine,yfine,'r-',lw=3)
  ymin = yfine.min()
  if yfine.min()>0:
    ymin = yfine.min()*0.8
  ymax = yfine.max()*1.25
  plt.ylim(ymin,ymax)
  plt.ylabel('y = f(x)',fontsize=20)
  plt.xlabel('x',fontsize=20)
  plt.xticks(fontsize=20)
  plt.yticks(fontsize=20)


  if Nfunc == 1:
    plt.text(2,ymax*0.8,'f(x) = sin(x)*log(x)', fontsize=30)
  elif Nfunc == 2:
    plt.text(2,ymax*0.8,'f(x) = exp(sin(x)*log(x))', fontsize=30)
  elif Nfunc == 3:
    plt.text(2,ymax*0.8,'f(x) = cos(0.001*(sinh(x)*log(x)-1) + 5)', fontsize=30)
  else:
    plt.text(2,ymax*0.8,'f(x) = sinh(x)*log(x) -1', fontsize=30)

  plt.text(2,ymax*0.6, 'Integral = {:4.8f}'.format(integral), fontsize=40)
  plt.text( 8, ymax, 'Simpson\'s 1/3', fontsize=40)
  plt.subplot(212)

  integral=0
  x=np.linspace(1,10,N+1)
  for xi in x:
    if xi == x[-1]:
      continue
    h = (x[1]-x[0])*0.5
    x1 = xi
    x2 = xi+h
    xvalue= [x1,x2]
    yvalue = polynew([x1,x2],Nfunc) 
    integral += (1/2.)*( yvalue[0] + yvalue[1] )*h
    plt.scatter(xvalue,yvalue,s=300)
   # print(xvalue,yvalue)
    p_fitted = np.polynomial.Polynomial.fit(xvalue, yvalue, deg=1)
    c=p_fitted.convert().coef
    c=np.flip(c)
    xifine = np.linspace(x1,x2,100)
    yifine = np.polyval(c,xifine)
    plt.plot(xifine,yifine)

    x1 = xi+h
    x2 = xi+2*h
    xvalue= [x1,x2]
    yvalue = polynew([x1,x2],Nfunc) 
    integral += (1/2.)*( yvalue[0] + yvalue[1] )*h
    plt.scatter(xvalue,yvalue,s=300)
   # print(xvalue,yvalue)
    p_fitted = np.polynomial.Polynomial.fit(xvalue, yvalue, deg=1)
    c=p_fitted.convert().coef
    c=np.flip(c)
    xifine = np.linspace(x1,x2,100)
    yifine = np.polyval(c,xifine)
    plt.plot(xifine,yifine)



  xfine=np.linspace(1,10,91)
  yfine = polynew(xfine,Nfunc)
  plt.plot(xfine,yfine,'r-',lw=3)
  ymin = yfine.min()
  if yfine.min()>0:
    ymin = yfine.min()*0.8
  ymax = yfine.max()*1.25
  plt.ylim(ymin,ymax)
  plt.ylabel('y = f(x)',fontsize=20)
  plt.xlabel('x',fontsize=20)
  plt.xticks(fontsize=20)
  plt.yticks(fontsize=20)


  if Nfunc == 1:
    plt.text(2,ymax*0.8,'f(x) = sin(x)*log(x)', fontsize=30)
  elif Nfunc == 2:
    plt.text(2,ymax*0.8,'f(x) = exp(sin(x)*log(x))', fontsize=30)
  elif Nfunc == 3:
    plt.text(2,ymax*0.8,'f(x) = cos(0.001*(sinh(x)*log(x)-1) + 5)', fontsize=30)
  else:
    plt.text(2,ymax*0.8,'f(x) = sinh(x)*log(x) -1', fontsize=30)

  plt.text(2,ymax*0.6, 'Integral = {:4.8f}'.format(integral), fontsize=40)
  plt.text( 8, ymax, 'Trapezoid', fontsize=40)

interactive(children=(IntSlider(value=1, description='N', layout=Layout(width='60%'), max=20, min=1), IntSlide…

# Monte Carlo Integration

In [16]:
@interact(Nrandom=widgets.IntSlider(min=1, max=20000, step=100, value=1,layout=widgets.Layout(width='60%')) , 
          Nfunc=widgets.IntSlider(min=1, max=5, step=1, value=1,layout=widgets.Layout(width='60%')) , continuous_update=True )
def plot(Nrandom, Nfunc):
  N = Nrandom
  def polynew(x,Nfunc):
    if Nfunc == 1:
      return np.sin(x)*np.log(x)
    elif Nfunc == 2:
      return np.exp(np.sin(x)*np.log(x))
    elif Nfunc == 3:
      return np.cos(0.001*(np.sinh(x)*np.log(x)-1)+5)
    else:
      return np.sinh(x)*np.log(x)-1
  fig, axs = plt.subplots(2,1,figsize=(16,12
                              ))
  plt.subplot(211)

  integral=0
  x=(np.random.random(size=Nrandom)*9)+1
  yvalue = polynew(x, Nfunc)
  a,b = 1, 10
  integral = ((b-a)/(x.size))*np.sum(yvalue)


  xfine=np.linspace(1,10,91)
  yfine = polynew(xfine,Nfunc)
  plt.plot(xfine,yfine,'r-',lw=3)
  ymin = yfine.min()
  if yfine.min()>0:
    ymin = yfine.min()*0.8
  ymax = yfine.max()*1.25
  plt.ylim(ymin,ymax)
  plt.ylabel('y = f(x)',fontsize=35)
  plt.xlabel('x',fontsize=25)
  plt.xticks(fontsize=25)
  plt.yticks(fontsize=35)


  if Nfunc == 1:
    plt.text(2,ymax*0.8,'f(x) = sin(x)*log(x)', fontsize=30)
  elif Nfunc == 2:
    plt.text(2,ymax*0.8,'f(x) = exp(sin(x)*log(x))', fontsize=30)
  elif Nfunc == 3:
    plt.text(2,ymax*0.8,'f(x) = cos(0.001*(sinh(x)*log(x)-1) + 5)', fontsize=30)
  else:
    plt.text(2,ymax*0.8,'f(x) = sinh(x)*log(x) -1', fontsize=30)

  plt.text(2,ymax*0.6, 'Integral = {:4.8f}'.format(integral), fontsize=40)
  plt.text( 8, ymax, 'Monte Carlo integration', fontsize=40)

  plt.subplot(212)
  plt.hist((b-a)*yvalue)
  plt.xlabel( 'Values of (b-a)*f(xi)',fontsize=35 )
  plt.ylabel( 'Entries' ,fontsize=35)
  plt.xticks(fontsize=40)
  plt.yticks(fontsize=40)

interactive(children=(IntSlider(value=1, description='Nrandom', layout=Layout(width='60%'), max=20000, min=1, …